In [11]:
from folium import plugins , GeoJson
from folium import Icon, Element, CircleMarker, Map, Popup, LayerControl

import pandas as pd

In [12]:
df = pd.read_csv('../Data/modified_Thawra_2019-2020 - Main.csv', header=1, parse_dates=['Startdate'])

In [13]:
df = df[['Startdate','gpslatend', 'gpslongend','demandtypeone']].sample(100)
df

,Startdate,gpslatend,gpslongend,demandtypeone
2483,2020-01-06,33.987122,35.640022,Human rights / civic freedoms
2938,2020-01-31,33.563685,35.380445,Economic/Financial
2692,2020-01-16,34.121280,35.651951,Thawra 2019
2258,2019-12-19,33.546030,35.381849,Labor
2628,2020-01-15,33.821289,35.882362,Thawra 2019
...,...,...,...,...
822,2019-11-03,33.898737,35.477927,Thawra 2019
2693,2020-01-16,34.107844,35.652755,Thawra 2019
1033,2019-11-08,33.563231,35.378631,Thawra 2019
1010,2019-11-07,34.203283,35.842145,Thawra 2019


In [14]:
df = df.reset_index().drop(columns='index').dropna()
df

,Startdate,gpslatend,gpslongend,demandtypeone
0,2020-01-06,33.987122,35.640022,Human rights / civic freedoms
1,2020-01-31,33.563685,35.380445,Economic/Financial
2,2020-01-16,34.121280,35.651951,Thawra 2019
3,2019-12-19,33.546030,35.381849,Labor
4,2020-01-15,33.821289,35.882362,Thawra 2019
...,...,...,...,...
95,2019-11-03,33.898737,35.477927,Thawra 2019
96,2020-01-16,34.107844,35.652755,Thawra 2019
97,2019-11-08,33.563231,35.378631,Thawra 2019
98,2019-11-07,34.203283,35.842145,Thawra 2019


In [15]:
def color_demand(demand):
    d = {'Corruption':'red', 'Politics (local)':'blue', 'Thawra 2019':'green',
       'Economic/Financial':'purple', 'Human rights / civic freedoms':'orange', 'Social':'darkred',
       'Politics (national)':'lightred', "Women's rights":'pink',
       'Politics (regional/international)':'darkblue', 'Religion/Sectarian':'red',
       'Security':'darkgreen', 'Labor':'gray'}
    color= d[demand]
    return color
df['color'] = df['demandtypeone'].apply(color_demand)

In [16]:
color_demand('Politics (regional/international)')

'darkblue'

In [17]:
df

,Startdate,gpslatend,gpslongend,demandtypeone,color
0,2020-01-06,33.987122,35.640022,Human rights / civic freedoms,orange
1,2020-01-31,33.563685,35.380445,Economic/Financial,purple
2,2020-01-16,34.121280,35.651951,Thawra 2019,green
3,2019-12-19,33.546030,35.381849,Labor,gray
4,2020-01-15,33.821289,35.882362,Thawra 2019,green
...,...,...,...,...,...
95,2019-11-03,33.898737,35.477927,Thawra 2019,green
96,2020-01-16,34.107844,35.652755,Thawra 2019,green
97,2019-11-08,33.563231,35.378631,Thawra 2019,green
98,2019-11-07,34.203283,35.842145,Thawra 2019,green


In [18]:
# Create a map object
m = Map(location=[33.905074, 35.744929])

# Create the data for the TimestampedGeoJson plugin
data = {
    'type': 'FeatureCollection',
    'features': []
}

for i, row in df.iterrows():
    t, lat, lon, demand, color = row['Startdate'], row['gpslatend'], row['gpslongend'], row['demandtypeone'], row['color']
    # CircleMarker(location=[lat, lon],
    #              radius=5,
    #              color=color,
    #              popup=demand,
    #              fill_color=color,
    #              fill=True).add_to(m)
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [lon, lat],
        },
        'properties': {
            'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
            'value':demand
        }
    }
    data['features'].append(feature)

# Add a timeline to the map
timeline = plugins.TimestampedGeoJson(data, period='P1D', add_last_point=False).add_to(m)

# # Add a timeline to the map

# coordinates = []
# time = []
# for indx in df.index:
#     coordinates.append([df['gpslatend'][indx], df['gpslongend'][indx]])
#     time.append(df['Startdate'][indx].strftime('%Y-%m-%dT%H:%M:%S'))

# timeline = plugins.TimestampedGeoJson({
#     'type': 'FeatureCollection',
#     'features': [
#         {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'Point',
#                 'coordinates': coordinates,
#             },
#             'properties': {
#                 'time': time
#             }
#         }
#     ]
# }, period='PT1M', add_last_point=True).add_to(m)

# Display the map
m

In [19]:
# layers = {}
# for value in df['value'].unique():
#     # Create the data for the TimestampedGeoJson plugin
#     data = {
#         'type': 'FeatureCollection',
#         'features': []
#     }
#     for i, row in df[df['value'] == value].iterrows():
#         lon, lat, t, value = row['lon'], row['lat'], row['time'], row['value']
#         feature = {
#             'type': 'Feature',
#             'geometry': {
#                 'type': 'Point',
#                 'coordinates': [lon, lat],
#             },
#             'properties': {
#                 'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
#                 'value': value
#             }
#         }
#         data['features'].append(feature)


In [21]:
# Create a map object

m2 = Map(location=[33.905074, 35.744929])

# Create a list of GeoJson objects, one for each value

geojson_list = []

for demand in df['demandtypeone'].unique():
    # Create the data for the GeoJson object
    data = {
        'type': 'FeatureCollection',
        'features': []
    }
    for i, row in df[df['demandtypeone'] == demand].iterrows():
        lon, lat, t, color = row['gpslongend'], row['gpslatend'], row['Startdate'], row['color']
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',  # Set the type to something other than 'point'
                'coordinates': [lon, lat],  # Set the coordinates of the feature
            },
            'properties': {
                'time': t.strftime('%Y-%m-%dT%H:%M:%S'),
                'color': color,
                'value':demand
            }
        }
        # Add a popup to the feature
        popup = Popup(demand)
        feature['properties']['popup'] = str(popup)
        data['features'].append(feature)



    # Create a GeoJson object and add it to the map
    def style_function(feature):
        color = feature['properties']['color']
        return {'color': color}

    geojson = GeoJson(data, style_function=style_function).add_to(m2)
   
    geojson_list.append(geojson)

# LayerControl(position='topright', collapsed=False).add_to(m2)

timeline = plugins.TimestampedGeoJson(data, period='P1D', add_last_point=True).add_to(m2)

# Display the map
m2